In [1]:
# 이거 실행하고 다시 런타임 시작
# 한글 폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (9,103 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from pykrx import stock
sns.set(font='NaNumBarunGothic')


import warnings
warnings.filterwarnings('ignore')
import re
pd.options.display.max_columns = None

In [4]:
# 기준이 되는 재무제표 데이터 끌어오기(여기에 있는 종목만 선정하기 위해)
fs = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/real_real_df.csv')

# 종목코드 6자리로 채워주기
fs['Stock'] = fs['Stock'].astype('str')
fs['Stock'] = fs['Stock'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

stock_code = list(fs['Stock'])

In [6]:
# altman k - score

altman_k_data = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/altman_k_score_data.csv')
altman_k_data.head(10)

,Massive Download,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Report as of : 2022-10-28,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Portfolio - kosdaq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,KRW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,2016/Annual,2016/Annual,2016/Annual,2016/Annual,2017/Annual,2017/Annual,2017/Annual,2017/Annual,2018/Annual,2018/Annual,2018/Annual,2018/Annual,2019/Annual,2019/Annual,2019/Annual,2019/Annual,2020/Annual,2020/Annual,2020/Annual,2020/Annual,2021/Annual,2021/Annual,2021/Annual,2021/Annual
5,KIS,Stock,Name,자산총계/015000,자본총계/018900,총자본회전율/094010,이익잉여금/018300,자산총계/015000,자본총계/018900,총자본회전율/094010,이익잉여금/018300,자산총계/015000,자본총계/018900,총자본회전율/094010,이익잉여금/018300,자산총계/015000,자본총계/018900,총자본회전율/094010,이익잉여금/018300,자산총계/015000,자본총계/018900,총자본회전율/094010,이익잉여금/018300,자산총계/015000,자본총계/018900,총자본회전율/094010,이익잉여금/018300
6,036644,389500,(주)에스비비테크,"16,847,204,000.","7,988,868,000.",0.51,"6,846,199,000.","16,925,578,000.","8,256,747,000.",0.53,"7,114,078,000.","24,717,222,000.","4,999,033,000.",0.44,"-966,592,000.","19,486,814,000.","209,907,000.",0.27,"-5,755,717,000.","14,990,522,000.","1,778,683,000.",0.38,"-8,866,735,000.","17,631,330,000.","6,595,519,000.",0.42,"-11,682,659,000."
7,PM6283,441270,(주)파인엠텍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,497730,296640,(주)이노룰스,"7,777,259,000.","1,304,510,000.",1.43,"-239,992,000.","8,788,407,000.","2,077,612,000.",1.20,"528,415,000.","15,980,953,000.","6,544,978,000.",0.75,"926,743,000.","15,788,912,000.","8,680,699,000.",0.84,"1,545,436,000.","13,005,199,000.","7,082,073,000.",0.74,"3,136,747,000.","17,530,505,000.","13,048,284,000.",1.05,"6,069,333,000."
9,499459,417970,모델솔루션(주),"34,617,716,000.","14,238,164,000.",1.40,"11,548,021,000.","33,358,522,000.","13,674,786,000.",1.49,"10,984,643,000.","41,207,294,000.","19,754,083,000.",1.49,"17,063,940,000.","45,289,359,000.","24,595,415,000.",1.40,"21,905,272,000.","42,338,149,000.","22,751,342,000.",1.26,"20,061,199,000.","48,265,607,000.","27,033,951,000.",1.33,"24,343,808,000."


In [17]:
# altman k - score

altman_k_data = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/altman_k_score_data.csv')

def make_preprocessing(df):
  col_name = ['KIS', 'Stock', 'Name']
  for i in df.columns[3:]:
    col_name.append(f"{df[i].iloc[5].split('/')[0]} ({df[i].iloc[4].split('/')[0]})")

  df.columns = col_name
  df.drop([0,1,2,3,4,5], axis=0, inplace=True)
  df = df.reset_index(drop=True)

  # 뒤에 널값인 데이터도 같이 들어왔는데 같이 삭제
  last_index = df[df['Name']=='(주)아바텍'].index # 데이터가 들어있는 맨 마지막 인덱스. 이 이후꺼를 삭제시키면 된다.
  df = df.iloc[:list(df[df['Name']=='(주)아바텍'].index).pop()+1, :]

  # stock code가 없는 기업 삭제
  del_index = list(df[df['Stock'].isnull()].index)
  df.drop(del_index, axis=0, inplace=True)
  df = df.reset_index(drop = True)

  # 인수합병된 기업들 빼기
  df['인수합병여부'] = 0
  df['인수합병여부'] = df.apply(lambda x:1 if '인수목적' in x['Name'] else 0,axis=1)

  df = df[df['인수합병여부']==0]
  df.drop('인수합병여부', axis=1, inplace =True)
  df = df.reset_index(drop=True)

  # 문자열 float로 변경
  for i in df.columns[3:]:
    df[i] = df[i].astype('str')
    df[i] = df[i].apply(lambda x: re.sub(',', '', x))
    df[i] = df[i].astype('float')

  return df

  # null인 경우 0으로 채운다. -> 이거 잠깐 보류

altman_k = make_preprocessing(altman_k_data)
altman_k

,KIS,Stock,Name,자산총계 (2016),자본총계 (2016),총자본회전율 (2016),이익잉여금 (2016),자산총계 (2017),자본총계 (2017),총자본회전율 (2017),이익잉여금 (2017),자산총계 (2018),자본총계 (2018),총자본회전율 (2018),이익잉여금 (2018),자산총계 (2019),자본총계 (2019),총자본회전율 (2019),이익잉여금 (2019),자산총계 (2020),자본총계 (2020),총자본회전율 (2020),이익잉여금 (2020),자산총계 (2021),자본총계 (2021),총자본회전율 (2021),이익잉여금 (2021)
0,036644,389500,(주)에스비비테크,1.684720e+10,7.988868e+09,0.51,6.846199e+09,1.692558e+10,8.256747e+09,0.53,7.114078e+09,2.471722e+10,4.999033e+09,0.44,-9.665920e+08,1.948681e+10,2.099070e+08,0.27,-5.755717e+09,1.499052e+10,1.778683e+09,0.38,-8.866735e+09,1.763133e+10,6.595519e+09,0.42,-1.168266e+10
1,PM6283,441270,(주)파인엠텍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,497730,296640,(주)이노룰스,7.777259e+09,1.304510e+09,1.43,-2.399920e+08,8.788407e+09,2.077612e+09,1.20,5.284150e+08,1.598095e+10,6.544978e+09,0.75,9.267430e+08,1.578891e+10,8.680699e+09,0.84,1.545436e+09,1.300520e+10,7.082073e+09,0.74,3.136747e+09,1.753050e+10,1.304828e+10,1.05,6.069333e+09
3,499459,417970,모델솔루션(주),3.461772e+10,1.423816e+10,1.40,1.154802e+10,3.335852e+10,1.367479e+10,1.49,1.098464e+10,4.120729e+10,1.975408e+10,1.49,1.706394e+10,4.528936e+10,2.459542e+10,1.40,2.190527e+10,4.233815e+10,2.275134e+10,1.26,2.006120e+10,4.826561e+10,2.703395e+10,1.33,2.434381e+10
4,HV2035,393890,더블유씨피(주),1.092831e+10,1.092831e+10,NaN,-1.469100e+07,9.287483e+10,8.748688e+10,NaN,-1.900798e+09,1.577923e+11,1.472231e+11,NaN,-1.140166e+10,4.307295e+11,1.939382e+11,0.12,-1.754902e+10,5.231622e+11,1.633585e+11,0.23,-4.812881e+10,6.300173e+11,4.057576e+11,0.32,-5.814062e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,I73639,950110,SBI핀테크솔루션즈(주),3.042943e+10,1.912966e+10,0.58,3.320965e+09,3.053952e+10,1.931119e+10,0.70,3.858105e+09,7.522338e+10,3.436309e+10,0.45,5.602427e+09,7.549365e+10,3.563226e+10,0.28,1.978127e+09,1.318126e+11,4.277608e+10,0.29,5.528564e+09,2.351104e+11,4.145961e+10,0.16,8.382701e+09
1441,125028,046970,(주)우리로,5.717900e+10,3.268302e+10,0.31,-8.400150e+08,5.066045e+10,3.181893e+10,0.31,-1.103422e+10,7.200770e+10,3.462791e+10,0.52,-1.232072e+10,7.169981e+10,2.801872e+10,1.41,-2.311653e+10,5.813646e+10,1.945891e+10,1.50,-3.459564e+10,5.986624e+10,3.890705e+10,1.64,-2.151513e+10
1442,383046,113810,디젠스(주),9.354566e+10,3.656179e+10,0.70,1.406798e+10,7.913090e+10,2.505458e+10,0.61,2.138022e+09,6.889130e+10,2.659972e+10,0.50,-9.085789e+09,7.071857e+10,2.917334e+10,0.57,-6.512168e+09,6.336970e+10,2.121784e+10,0.74,-1.446767e+10,6.245524e+10,2.026761e+10,0.78,-1.541790e+10
1443,F11411,155650,와이엠씨(주),9.460893e+10,5.202305e+10,1.59,3.491621e+10,2.209364e+11,7.566230e+10,0.92,4.871990e+10,1.715772e+11,8.593090e+10,1.18,6.041253e+10,1.246931e+11,8.008635e+10,1.10,5.404676e+10,1.079085e+11,8.450543e+10,1.25,6.158887e+10,1.129362e+11,9.116672e+10,1.62,6.776209e+10


In [18]:
altman_k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   KIS            1445 non-null   object 
 1   Stock          1445 non-null   object 
 2   Name           1445 non-null   object 
 3   자산총계 (2016)    1360 non-null   float64
 4   자본총계 (2016)    1360 non-null   float64
 5   총자본회전율 (2016)  1326 non-null   float64
 6   이익잉여금 (2016)   1360 non-null   float64
 7   자산총계 (2017)    1388 non-null   float64
 8   자본총계 (2017)    1388 non-null   float64
 9   총자본회전율 (2017)  1362 non-null   float64
 10  이익잉여금 (2017)   1388 non-null   float64
 11  자산총계 (2018)    1401 non-null   float64
 12  자본총계 (2018)    1401 non-null   float64
 13  총자본회전율 (2018)  1375 non-null   float64
 14  이익잉여금 (2018)   1401 non-null   float64
 15  자산총계 (2019)    1422 non-null   float64
 16  자본총계 (2019)    1422 non-null   float64
 17  총자본회전율 (2019)  1393 non-null   float64
 18  이익잉여금 (2

In [19]:
# fs 데이터에 있는 종목코드만 대상으로 진행하기
tmp = []
altman_k_df = pd.DataFrame()
for i in altman_k['Stock'].unique():
  if i in fs['Stock'].unique():
    idx = list(altman_k[altman_k['Stock']==i].index.values).pop()
    tmp.append(idx)
altman_k_df  =pd.DataFrame(altman_k, index = tmp).reset_index(drop=True)

In [20]:
altman_k_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   KIS            1279 non-null   object 
 1   Stock          1279 non-null   object 
 2   Name           1279 non-null   object 
 3   자산총계 (2016)    1264 non-null   float64
 4   자본총계 (2016)    1264 non-null   float64
 5   총자본회전율 (2016)  1250 non-null   float64
 6   이익잉여금 (2016)   1264 non-null   float64
 7   자산총계 (2017)    1279 non-null   float64
 8   자본총계 (2017)    1279 non-null   float64
 9   총자본회전율 (2017)  1278 non-null   float64
 10  이익잉여금 (2017)   1279 non-null   float64
 11  자산총계 (2018)    1279 non-null   float64
 12  자본총계 (2018)    1279 non-null   float64
 13  총자본회전율 (2018)  1279 non-null   float64
 14  이익잉여금 (2018)   1279 non-null   float64
 15  자산총계 (2019)    1279 non-null   float64
 16  자본총계 (2019)    1279 non-null   float64
 17  총자본회전율 (2019)  1279 non-null   float64
 18  이익잉여금 (2

# 결측치 처리

In [36]:
# 엔바이오니아 2020년 이익잉여금 결측
altman_k_df[altman_k_df['이익잉여금 (2020)'].isnull()]

,KIS,Stock,Name,자산총계 (2016),자본총계 (2016),총자본회전율 (2016),이익잉여금 (2016),자산총계 (2017),자본총계 (2017),총자본회전율 (2017),이익잉여금 (2017),자산총계 (2018),자본총계 (2018),총자본회전율 (2018),이익잉여금 (2018),자산총계 (2019),자본총계 (2019),총자본회전율 (2019),이익잉여금 (2019),자산총계 (2020),자본총계 (2020),총자본회전율 (2020),이익잉여금 (2020),자산총계 (2021),자본총계 (2021),총자본회전율 (2021),이익잉여금 (2021),2018_X1,2018_X2,2018_X3,2018_X4,2018_K,2019_X1,2019_X2,2019_X3,2019_X4,2019_K,2020_X1,2020_X2,2020_X3,2020_X4,2020_K,2021_X1,2021_X2,2021_X3,2021_X4,2021_K
125,187195,317870,(주)엔바이오니아,1.669736e+10,8.512218e+09,0.35,-2.392198e+09,1.587522e+10,3.863269e+09,0.49,-6.279575e+09,1.802743e+10,5.674275e+09,0.5,-5.850928e+09,3.014196e+10,2.155320e+10,0.4,-4.913816e+09,2.923212e+10,2.004812e+10,0.28,NaN,3.628851e+10,1.801583e+10,0.23,NaN,23.61516,-0.693147,-0.324557,0.314758,10.452728,24.129184,-0.916291,-0.163022,0.715057,13.534654,24.098534,-1.272966,NaN,0.685825,NaN,24.314767,-1.469676,NaN,0.496461,NaN


In [37]:
# 엔바이오니아 2021년 이익잉여금 결측
altman_k_df[altman_k_df['이익잉여금 (2021)'].isnull()]

,KIS,Stock,Name,자산총계 (2016),자본총계 (2016),총자본회전율 (2016),이익잉여금 (2016),자산총계 (2017),자본총계 (2017),총자본회전율 (2017),이익잉여금 (2017),자산총계 (2018),자본총계 (2018),총자본회전율 (2018),이익잉여금 (2018),자산총계 (2019),자본총계 (2019),총자본회전율 (2019),이익잉여금 (2019),자산총계 (2020),자본총계 (2020),총자본회전율 (2020),이익잉여금 (2020),자산총계 (2021),자본총계 (2021),총자본회전율 (2021),이익잉여금 (2021),2018_X1,2018_X2,2018_X3,2018_X4,2018_K,2019_X1,2019_X2,2019_X3,2019_X4,2019_K,2020_X1,2020_X2,2020_X3,2020_X4,2020_K,2021_X1,2021_X2,2021_X3,2021_X4,2021_K
125,187195,317870,(주)엔바이오니아,1.669736e+10,8.512218e+09,0.35,-2.392198e+09,1.587522e+10,3.863269e+09,0.49,-6.279575e+09,1.802743e+10,5.674275e+09,0.5,-5.850928e+09,3.014196e+10,2.155320e+10,0.4,-4.913816e+09,2.923212e+10,2.004812e+10,0.28,NaN,3.628851e+10,1.801583e+10,0.23,NaN,23.61516,-0.693147,-0.324557,0.314758,10.452728,24.129184,-0.916291,-0.163022,0.715057,13.534654,24.098534,-1.272966,NaN,0.685825,NaN,24.314767,-1.469676,NaN,0.496461,NaN


125

In [41]:
# 널값 채우기
altman_k_df.loc[list(altman_k_df[altman_k_df['이익잉여금 (2021)'].isnull()].index).pop(), '이익잉여금 (2020)'] = 0
altman_k_df.loc[list(altman_k_df[altman_k_df['이익잉여금 (2021)'].isnull()].index).pop(), '이익잉여금 (2021)'] = 12858000

In [42]:
altman_k_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 47 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   KIS            1279 non-null   object 
 1   Stock          1279 non-null   object 
 2   Name           1279 non-null   object 
 3   자산총계 (2016)    1264 non-null   float64
 4   자본총계 (2016)    1264 non-null   float64
 5   총자본회전율 (2016)  1250 non-null   float64
 6   이익잉여금 (2016)   1264 non-null   float64
 7   자산총계 (2017)    1279 non-null   float64
 8   자본총계 (2017)    1279 non-null   float64
 9   총자본회전율 (2017)  1278 non-null   float64
 10  이익잉여금 (2017)   1279 non-null   float64
 11  자산총계 (2018)    1279 non-null   float64
 12  자본총계 (2018)    1279 non-null   float64
 13  총자본회전율 (2018)  1279 non-null   float64
 14  이익잉여금 (2018)   1279 non-null   float64
 15  자산총계 (2019)    1279 non-null   float64
 16  자본총계 (2019)    1279 non-null   float64
 17  총자본회전율 (2019)  1279 non-null   float64
 18  이익잉여금 (2

# 알트만 K score 도출

In [43]:
def altman_k_score(df):
  year = ['2018', '2019', '2020', '2021']
  for y in year:

    for col in df.columns:
      if y in col:
        if '자산총계' in col:
          자산총계 = col
        if '총자본회전율' in col:
          총자본회전율 = col
        if '이익잉여금' in col:
          이익잉여금 = col
        if '자산총계' in col:
          자산총계 = col
        if '자본총계' in col:
          자본총계 = col
        
    df[f'{y}_X1'] = np.log(df[f'{자산총계}'])
    df[f'{y}_X2'] = np.log(df[f'{총자본회전율}'])
    df[f'{y}_X3'] = df[f'{이익잉여금}'] / df[f'{자산총계}']
    df[f'{y}_X4'] = df[f'{자본총계}'] / df[f'{자산총계}']

    df[f'{y}_K'] = -17.862 + 1.472 * df[f'{y}_X1'] + 3.041 * df[f'{y}_X2'] + 14.839 * df[f'{y}_X3'] + 1.516 * df[f'{y}_X4']
  return df


altman_k_df2 = altman_k_score(altman_k_df)

In [44]:
altman_k_df2

,KIS,Stock,Name,자산총계 (2016),자본총계 (2016),총자본회전율 (2016),이익잉여금 (2016),자산총계 (2017),자본총계 (2017),총자본회전율 (2017),이익잉여금 (2017),자산총계 (2018),자본총계 (2018),총자본회전율 (2018),이익잉여금 (2018),자산총계 (2019),자본총계 (2019),총자본회전율 (2019),이익잉여금 (2019),자산총계 (2020),자본총계 (2020),총자본회전율 (2020),이익잉여금 (2020),자산총계 (2021),자본총계 (2021),총자본회전율 (2021),이익잉여금 (2021),2018_X1,2018_X2,2018_X3,2018_X4,2018_K,2019_X1,2019_X2,2019_X3,2019_X4,2019_K,2020_X1,2020_X2,2020_X3,2020_X4,2020_K,2021_X1,2021_X2,2021_X3,2021_X4,2021_K
0,036644,389500,(주)에스비비테크,1.684720e+10,7.988868e+09,0.51,6.846199e+09,1.692558e+10,8.256747e+09,0.53,7.114078e+09,2.471722e+10,4.999033e+09,0.44,-9.665920e+08,1.948681e+10,2.099070e+08,0.27,-5.755717e+09,1.499052e+10,1.778683e+09,0.38,-8.866735e+09,1.763133e+10,6.595519e+09,0.42,-1.168266e+10,23.930766,-0.820981,-0.039106,0.202249,14.593801,23.693004,-1.309333,-0.295365,0.010772,8.665832,23.430684,-0.967584,-0.591489,0.118654,5.088312,23.592943,-0.867501,-0.662608,0.374079,4.963409
1,497730,296640,(주)이노룰스,7.777259e+09,1.304510e+09,1.43,-2.399920e+08,8.788407e+09,2.077612e+09,1.20,5.284150e+08,1.598095e+10,6.544978e+09,0.75,9.267430e+08,1.578891e+10,8.680699e+09,0.84,1.545436e+09,1.300520e+10,7.082073e+09,0.74,3.136747e+09,1.753050e+10,1.304828e+10,1.05,6.069333e+09,23.494663,-0.287682,0.057990,0.409549,17.328700,23.482574,-0.174353,0.097881,0.549797,18.460090,23.288615,-0.301105,0.241192,0.544557,19.907774,23.587208,0.048790,0.346216,0.744319,23.272621
2,499459,417970,모델솔루션(주),3.461772e+10,1.423816e+10,1.40,1.154802e+10,3.335852e+10,1.367479e+10,1.49,1.098464e+10,4.120729e+10,1.975408e+10,1.49,1.706394e+10,4.528936e+10,2.459542e+10,1.40,2.190527e+10,4.233815e+10,2.275134e+10,1.26,2.006120e+10,4.826561e+10,2.703395e+10,1.33,2.434381e+10,24.441881,0.398776,0.414100,0.479383,26.200702,24.536338,0.336472,0.483674,0.543073,27.279234,24.468954,0.231112,0.473833,0.537372,26.704971,24.599985,0.285179,0.504372,0.560108,27.549903
3,HS8995,314140,(주)알피바이오,5.795457e+10,3.414006e+10,0.45,1.569932e+09,6.457702e+10,4.002163e+10,1.11,7.451501e+09,8.959963e+10,5.270432e+10,0.84,1.181495e+10,1.159244e+11,5.672861e+10,0.68,1.523828e+10,1.304471e+11,6.103534e+10,0.79,2.028269e+10,1.437370e+11,6.492697e+10,0.84,2.415975e+10,25.218617,-0.174353,0.131864,0.588220,21.578064,25.476204,-0.385662,0.131450,0.489358,21.158629,25.594233,-0.235722,0.155486,0.467893,22.112463,25.691251,-0.174353,0.168083,0.451707,22.604285
4,867055,129920,(주)대성하이텍,1.062744e+11,4.167473e+10,0.64,1.542425e+10,1.143085e+11,3.729471e+10,0.75,1.104422e+10,1.238696e+11,4.710453e+10,0.74,1.092001e+10,1.264083e+11,5.137788e+10,0.58,2.519386e+10,1.182687e+11,3.156177e+10,0.59,4.876566e+09,1.427651e+11,3.098168e+10,0.79,2.198118e+10,25.542495,-0.301105,0.088157,0.380275,20.705556,25.562783,-0.544727,0.199305,0.406444,21.683563,25.496225,-0.527633,0.041233,0.266865,19.080335,25.684466,-0.235722,0.153967,0.217012,21.842416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,I73639,950110,SBI핀테크솔루션즈(주),3.042943e+10,1.912966e+10,0.58,3.320965e+09,3.053952e+10,1.931119e+10,0.70,3.858105e+09,7.522338e+10,3.436309e+10,0.45,5.602427e+09,7.549365e+10,3.563226e+10,0.28,1.978127e+09,1.318126e+11,4.277608e+10,0.29,5.528564e+09,2.351104e+11,4.145961e+10,0.16,8.382701e+09,25.043728,-0.798508,0.074477,0.456814,18.371803,25.047314,-1.272966,0.026203,0.471990,16.240915,25.604647,-1.237874,0.041943,0.324522,17.178026,26.183321,-1.832581,0.035654,0.176341,15.903376
1275,125028,046970,(주)우리로,5.717900e+10,3.268302e+10,0.31,-8.400150e+08,5.066045e+10,3.181893e+10,0.31,-1.103422e+10,7.200770e+10,3.462791e+10,0.52,-1.232072e+10,7.169981e+10,2.801872e+10,1.41,-2.311653e+10,5.813646e+10,1.945891e+10,1.50,-3.459564e+10,5.986624e+10,3.890705e+10,1.64,-2.151513e+10,25.000039,-0.653926,-0.171103,0.480892,15.139504,24.995754,0.343590,-0.322407,0.390778,15.784827,24.786059,0.405465,-0.595076,0.334711,11.5

In [45]:
altman_k_df2['2018_K'].describe()

count    1279.000000
mean       21.587169
std         8.734610
min       -44.102633
25%        18.388053
50%        23.590757
75%        26.863582
max        55.431036
Name: 2018_K, dtype: float64

In [46]:
altman_k_df2['2019_K'].describe()

count    1279.000000
mean       21.108079
std        11.617042
min      -238.126049
25%        18.336452
50%        23.282472
75%        26.753216
max        52.782036
Name: 2019_K, dtype: float64

In [47]:
altman_k_df2['2020_K'].describe()

count    1279.000000
mean       21.002325
std         9.795889
min       -73.117788
25%        17.530766
50%        23.114309
75%        26.984318
max        63.594235
Name: 2020_K, dtype: float64

In [48]:
altman_k_df2['2021_K'].describe()

count    1279.000000
mean       21.264215
std         9.263124
min       -57.796296
25%        17.897612
50%        23.197948
75%        27.102566
max        51.539606
Name: 2021_K, dtype: float64

# 부실 라벨링
- 0.75 > K : 부실가능성 x
- -2 < K < 0.75 : 판정 유보
- K < -2 : 부실가능성 심각

In [49]:
# 연도별 부실가능성 심각한 기업
print(len(altman_k_df2[altman_k_df2['2018_K']<-2]))
print(len(altman_k_df2[altman_k_df2['2019_K']<-2]))
print(len(altman_k_df2[altman_k_df2['2020_K']<-2]))
print(len(altman_k_df2[altman_k_df2['2021_K']<-2]))


30
42
37
39


In [51]:
# 라벨링
altman_k_df2['2018_부실여부판단'] = altman_k_df2.apply(lambda x: '안전기업' if x['2018_K']>0.75 else '판정유보기업' if x['2018_K'] > -2 else '부실 가능성 높은 기업', axis=1)
altman_k_df2['2019_부실여부판단'] = altman_k_df2.apply(lambda x: '안전기업' if x['2019_K']>0.75 else '판정유보기업' if x['2019_K'] > -2 else '부실 가능성 높은 기업', axis=1)
altman_k_df2['2020_부실여부판단'] = altman_k_df2.apply(lambda x: '안전기업' if x['2020_K']>0.75 else '판정유보기업' if x['2020_K'] > -2 else '부실 가능성 높은 기업', axis=1)
altman_k_df2['2021_부실여부판단'] = altman_k_df2.apply(lambda x: '안전기업' if x['2021_K']>0.75 else '판정유보기업' if x['2021_K'] > -2 else '부실 가능성 높은 기업', axis=1)

In [52]:
print(len(altman_k_df2[altman_k_df2['2018_K'].isnull()]))
print(len(altman_k_df2[altman_k_df2['2019_K'].isnull()]))
print(len(altman_k_df2[altman_k_df2['2020_K'].isnull()]))
print(len(altman_k_df2[altman_k_df2['2021_K'].isnull()]))

0
0
0
0


In [53]:
final_altman_k = altman_k_df2[['Stock', '2018_K', '2019_K', '2020_K', '2021_K', '2018_부실여부판단', '2019_부실여부판단', '2020_부실여부판단', '2021_부실여부판단']]

In [56]:
print(final_altman_k['2018_부실여부판단'].value_counts())
print(final_altman_k['2019_부실여부판단'].value_counts())
print(final_altman_k['2020_부실여부판단'].value_counts())
print(final_altman_k['2021_부실여부판단'].value_counts())

안전기업            1237
부실 가능성 높은 기업      30
판정유보기업            12
Name: 2018_부실여부판단, dtype: int64
안전기업            1229
부실 가능성 높은 기업      42
판정유보기업             8
Name: 2019_부실여부판단, dtype: int64
안전기업            1234
부실 가능성 높은 기업      37
판정유보기업             8
Name: 2020_부실여부판단, dtype: int64
안전기업            1232
부실 가능성 높은 기업      39
판정유보기업             8
Name: 2021_부실여부판단, dtype: int64


In [57]:
final_altman_k.to_csv('altman_k_score.csv', index = False)